In [1]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
import datetime
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()


Conexion exitosa con la Base de datos:irvin_hisminsa


In [2]:
def consulta_sql(conn):
   

    sql = conn.df(""" select
	mhtd.abrev_tipo_doc ,
	nt.numero_documento,
	mp.fecha_nacimiento ,
	mp.genero ,
	nt.anio_actual_paciente ,
	nt.mes_actual_paciente ,
	nt.dia_actual_paciente ,
	nt.fecha_atencion ,
	nt.codigo_item,
	nt.lab1 ,
	row_number() over (partition by nt.id_tipo_documento ,
	nt.numero_documento
order by
		nt.fecha_atencion asc) as rn
from
	maestros.nominal_trama2 nt
inner join maestros.maestro_paciente mp on
	mp.id_paciente = nt.id_paciente
inner join maestros.maestro_his_tipo_doc mhtd on
	mhtd.id_tipo_documento = nt.id_tipo_documento
where
		nt.anio_actual_paciente<3
	and nt.codigo_item like '9938%'""")
    return sql
df2=consulta_sql(conn)

d:\Irvin\Irvin\Python\pandas\modulos\ninio\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [37]:
df=df2
df['fecha_atencion'] = pd.to_datetime(df['fecha_atencion'])
#df['visita_count']  = df.groupby(['abrev_tipo_doc', 'numero_documento']).cumcount()
pivot_df = df.pivot(index=['abrev_tipo_doc', 'numero_documento'], columns='rn', values='fecha_atencion')

for i in range(2, len(pivot_df.columns) + 1):
    pivot_df[f'diff_{i}'] = (pivot_df[i] - pivot_df[1]).dt.days
    pivot_df[f'diff_{i}'].fillna(0, inplace=True)

#pivot_df.head()
pivot_df.to_excel("demo.xlsx", index=True)
#print(df_pivot[2])